# 환경변수 로딩

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# 벡터DB 클라이언트 초기화
- pinecone 객체 생성

In [2]:
from pinecone import Pinecone

pc = Pinecone(
    api_key=PINECONE_API_KEY
)


In [3]:
pc.list_indexes()

[
    {
        "name": "embedding-test",
        "metric": "cosine",
        "host": "embedding-test-r8d3erp.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "wiki",
        "metric": "cosine",
        "host": "wiki-r8d3erp.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

# index 객체 생성
- index 객체: embedding vector 를 저장하는 그릇 같은 것
- 유사도 기반의 검색 수행 

In [5]:
index_name = "wiki"

index = pc.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki_ns1': {'vector_count': 1659}},
 'total_vector_count': 1659,
 'vector_type': 'dense'}

# 검색하기
- Vector DB를 통해 질문에 대한 유사도가 가장 높은 top-k문서를 검색

## 방법1 Pinecone API 사용

### 질문 embeddingvect로 만들기

In [7]:
from langchain_openai import OpenAIEmbeddings
# embedding rorcp todtjd
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
)
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000024F71267320>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000024F740ABD70>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [12]:
# embedding vector로 변환
# question = ["김대중 대통령의 행적에 대해 알여줘"]
question = ["백남준에 대해 알려줘"]

embedded_question = embedding.embed_documents(
    question
)
embedded_question

[[-0.007300777360796928,
  -0.020231176167726517,
  -0.044150419533252716,
  0.02062126062810421,
  0.0007663156138733029,
  -0.01897226832807064,
  -0.03939848765730858,
  0.014743402600288391,
  -0.013067814521491528,
  -0.024309327825903893,
  -0.022607142105698586,
  0.0351075604557991,
  -0.008085377514362335,
  -0.0048405881971120834,
  -0.024965377524495125,
  -0.00037927913945168257,
  -0.03766084089875221,
  -0.021951092407107353,
  0.05376067012548447,
  0.003639306640252471,
  0.03301529213786125,
  -0.015851596370339394,
  -0.0018118959851562977,
  0.01757151260972023,
  -0.02953999862074852,
  -0.005248403176665306,
  0.057342350482940674,
  -0.049576133489608765,
  0.034327395260334015,
  -0.018475797027349472,
  -0.003805535612627864,
  -0.03594092279672623,
  0.033139411360025406,
  -0.037377141416072845,
  -0.011374495923519135,
  0.007952394895255566,
  0.0148852514103055,
  -0.034362856298685074,
  -0.010514537803828716,
  -0.019273696467280388,
  0.03232378140091896

In [13]:
len(embedded_question[0])

1536

- pincone'wiki' 'indexdml '

In [17]:
query_result = index.query(
    namespace="wiki_ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=3,  # 상위 3개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
)
print(query_result )

{'matches': [{'id': '4e450c0a-09a7-4ece-9a01-cca5e8dd1618',
              'metadata': {'chunk_id': 0.0,
                           'chunk_text': '백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 '
                                         '29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 '
                                         '예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).\n'
                                         '\n'
                                         '생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 '
                                         '여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 '
                                         "새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 "
                                         "아트의 창시자'로 알려져 있다.\n"
                                         '\n'
                                         '생애',
                           'title': '백남준',
                           'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80',
                           'wiki_id': '38'},
              'score': 0.544540405,
    

- 검색된 결과에서 필요한 데이터 추출하기

In [23]:
# query_result.matches
for r in query_result.matches:
    print(r.score, r.metadata)
    print("-"*30)

0.544540405 {'chunk_id': 0.0, 'chunk_text': "백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).\n\n생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.\n\n생애", 'title': '백남준', 'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80', 'wiki_id': '38'}
------------------------------
0.498300582 {'chunk_id': 39.0, 'chunk_text': '관련 영상 \n 백남준의 위성예술 1부 (1984년 방송)\n 백남준의 위성예술 2부 (1984년 방송)\n 백남준의 비디오아트세계 (1984년 방송)\n 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집\n\n각주\n\n외부 링크', 'title': '백남준', 'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80', 'wiki_id': '38'}
------------------------------
0.492431641 {'chunk_id': 40.0, 'chunk_text': '각주\n\n외부 링크 \n\n  백남준아트센터\n ARTCYCLOPEDIA에서 백남준 항목\n 백남준의 세계 (출처:Guggenheim.org)\n 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)\n 백남준 일대기 (출처:@ MedienKunstNetz)}\n 백남준 부고 "Father of Video Art Paik Nam-june Dies", (

In [24]:
# query_result.matches
for r in query_result.matches:
    # chunk 추출하기
    print(r.metadata)
    print("-"*30)

{'chunk_id': 0.0, 'chunk_text': "백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).\n\n생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.\n\n생애", 'title': '백남준', 'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80', 'wiki_id': '38'}
------------------------------
{'chunk_id': 39.0, 'chunk_text': '관련 영상 \n 백남준의 위성예술 1부 (1984년 방송)\n 백남준의 위성예술 2부 (1984년 방송)\n 백남준의 비디오아트세계 (1984년 방송)\n 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집\n\n각주\n\n외부 링크', 'title': '백남준', 'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80', 'wiki_id': '38'}
------------------------------
{'chunk_id': 40.0, 'chunk_text': '각주\n\n외부 링크 \n\n  백남준아트센터\n ARTCYCLOPEDIA에서 백남준 항목\n 백남준의 세계 (출처:Guggenheim.org)\n 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)\n 백남준 일대기 (출처:@ MedienKunstNetz)}\n 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)\n 백남준 부고 

In [25]:
for r in query_result.matches:
    # chunk 추출하기
    print(r.metadata["chunk_text"])
    print("-"*30)

백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).

생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.

생애
------------------------------
관련 영상 
 백남준의 위성예술 1부 (1984년 방송)
 백남준의 위성예술 2부 (1984년 방송)
 백남준의 비디오아트세계 (1984년 방송)
 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집

각주

외부 링크
------------------------------
각주

외부 링크 

  백남준아트센터
 ARTCYCLOPEDIA에서 백남준 항목
 백남준의 세계 (출처:Guggenheim.org)
 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)
 백남준 일대기 (출처:@ MedienKunstNetz)}
 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)
 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)
 백남준 공식 블로그
------------------------------


## 전체 코드 통합

In [42]:
# 전체 코드 통합, 함수로 정의
question = ["김대중 대통령의 행적에 대해 알려줘"]


from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

def search_top_k(question):
    # 환경변수 로딩
    load_dotenv(override=True)
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    
    #벡터db, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
        model = "text-embedding-3-small",
        api_key = OPENAI_API_KEY)
    
    
    # 질문을 embedding vector로 변경
    embedded_question = embedding.embed_documents(question)
    
    
    # top3 검색
    query_result = index.query(
        namespace="wiki_ns1",  # wiki-ns1 네임스페이스를 지정합니다.
        vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
        top_k=3,  # 상위 3개의 결과를 반환합니다.
        include_vector=False,  # 벡터를 포함하지 않습니다.
        include_metadata=True  # 메타데이터를 포함합니다.
    )
    
    
    # 결과에서 필요한 context 추출
    context = ""
    for r in query_result.matches:
        # chunk 추출하기
        # print(r.metadata["chunk_text"])
        # context = context + r.metadata["chunk_text"] + f'\n{"-"*30}\n'
        context = context + r.metadata["chunk_text"]
    return context
        

In [43]:
# 함수 호출
top_k_context = search_top_k(question)
print(top_k_context)

취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아1995년 부산 시장 선거 출마, 통합민주당 창당 활동에 합류
 1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령아니라, 그동안 여러분에게 지고 있었던 마음의 짐을 풀었다는 것이 가장 기쁩니다"라는 말로 과거의 일을 반성했다. 그리고 그 해 12월 18일, 김대중이 15대 대통령에 당선되면서 노무현은 사상 처음으로 여당에 몸담게 되었다.


## 방법2. LangChain API 사용
- 질문에 대한 유사도 검색 수행

In [12]:
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings


question = ["김대중 대통령의 행적에 대해 알려줘"]

In [13]:
# 환경변수 로딩
load_dotenv(override=True)
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']


In [14]:
#벡터db, index 객체 생성
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "wiki"
index = pc.Index(index_name)

In [15]:
# embedding 객체 생성
embedding = OpenAIEmbeddings(
model = "text-embedding-3-small",
api_key = OPENAI_API_KEY)

In [16]:
vector_store = PineconeVectorStore(index=index,        
                                   embedding=embedding, 
                                   text_key="chunk_text")

In [17]:
results = vector_store.similarity_search(
    query=question[0], 
    k=3, 
    namespace="wiki_ns1")

In [18]:
for result in results:
    print(result)
    # print(result.metadata)
    # print(result.page_content)
    print("-"*30)

page_content='취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아' metadata={'chunk_id': 59.0, 'title': '노무현', 'url': 'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%AC%B4%ED%98%84', 'wiki_id': '85'}
------------------------------
page_content='1995년 부산 시장 선거 출마, 통합민주당 창당 활동에 합류
 1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령' metadata={'chunk_id': 246.0, 'title': '노무현', 'url': 'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%AC%B4%ED%98%84', 'wiki_id': '85'}
------

## 전체 코드 통합

In [6]:
# 전체 코드 통합, 함수로 정의
question = "김대중 대통령의 행적에 대해 알려줘"


from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

def sm_search_top_k(question):
    # 환경변수 로딩
    load_dotenv(override=True)
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    
    #벡터db, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
        model = "text-embedding-3-small",
        api_key = OPENAI_API_KEY)
    
    
    # 질문을 embedding vector로 변경  <---  results = vector_store.similarity_search() 여기서 자동으로 됨
    # embedded_question = embedding.embed_documents(question)
    
    

    # top 3 검색
    # 텍스트 키는 "chunk_text"로 설정합니다.
    vector_store = PineconeVectorStore(index=index,        
                                       embedding=embedding, 
                                       text_key="chunk_text")  # default text 임, 우리가 설정한 값이 chunk_text
    
    # 질문에 대한 유사도 검색을 수행합니다.
    # query는 question 리스트의 첫 번째 요소를 사용하고, 상위 3개의 결과를 반환합니다.
    results = vector_store.similarity_search(
        query=question, # question 문자열을 embedding vector로 자동 변환됨
        k=3, 
        namespace="wiki_ns1")
    # print(results)
    
    # 검색된 문서의 메타데이터를 출력합니다.
    context = ""
    for result in results:
        # print(result)
        # print(result.metadata)
        # print(result.page_content)
        # print("-"*30)
        context = context + result.page_content
        
    return context

In [7]:
# 함수 호출
question = "김대중 대통령의 행적에 대해 알려줘"
top_k_context = sm_search_top_k(question)
print(top_k_context)

취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아1995년 부산 시장 선거 출마, 통합민주당 창당 활동에 합류
 1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령아니라, 그동안 여러분에게 지고 있었던 마음의 짐을 풀었다는 것이 가장 기쁩니다"라는 말로 과거의 일을 반성했다. 그리고 그 해 12월 18일, 김대중이 15대 대통령에 당선되면서 노무현은 사상 처음으로 여당에 몸담게 되었다.
